# Importing a library that is not in Colaboratory


In [ ]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install -U transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

# Load dataset

In [ ]:
from datasets import load_dataset

ds = load_dataset("uitnlp/vietnamese_students_feedback")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

vietnamese_students_feedback.py:   0%|          | 0.00/4.97k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/475k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/63.3k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/134k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11426 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1583 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3166 [00:00<?, ? examples/s]

# Show dataset

In [ ]:
print(ds)

emotion_labels = ds['train'].features['sentiment'].names

for i in range(5):
    text = ds['train'][i]['sentence']
    label_id = ds['train'][i]['sentiment']
    label_name = emotion_labels[label_id]
    print(f"\nTemplate {i+1}:")
    print("Content:", text)
    print("Emotion:", label_name)


DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 3166
    })
})

Template 1:
Content: slide giáo trình đầy đủ .
Emotion: positive

Template 2:
Content: nhiệt tình giảng dạy , gần gũi với sinh viên .
Emotion: positive

Template 3:
Content: đi học đầy đủ full điểm chuyên cần .
Emotion: negative

Template 4:
Content: chưa áp dụng công nghệ thông tin và các thiết bị hỗ trợ cho việc giảng dạy .
Emotion: negative

Template 5:
Content: thầy giảng bài hay , có nhiều bài tập ví dụ ngay trên lớp .
Emotion: positive


# Prepare Data

In [ ]:
from transformers import AutoTokenizer

# Use PhoBERT's tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

# Function to encode text
def tokenize(batch):
    return tokenizer(batch["sentence"], padding="max_length", truncation=True, max_length=128)

# Apply tokenizer to the entire dataset
tokenized_ds = ds.map(tokenize, batched=True)
tokenized_ds = tokenized_ds.rename_column("sentiment", "labels")  # Must use "labels" for Hugging Face Trainer
tokenized_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Map:   0%|          | 0/11426 [00:00<?, ? examples/s]

Map:   0%|          | 0/1583 [00:00<?, ? examples/s]

Map:   0%|          | 0/3166 [00:00<?, ? examples/s]

# Transfomers Model

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=8)

# Evaluate accuracy
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# Configure training settings
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

<ipython-input-5-de96abc8dbc6>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 522h0068 (522h0068-ton-duc-thang-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


TrainOutput(global_step=2145, training_loss=0.233894948359136, metrics={'train_runtime': 858.5874, 'train_samples_per_second': 39.924, 'train_steps_per_second': 2.498, 'total_flos': 2254851654782976.0, 'train_loss': 0.233894948359136, 'epoch': 3.0})

# Test

In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Move data to the same device as the model
    outputs = model(**inputs)
    probs = outputs.logits.softmax(dim=1)
    pred = probs.argmax().item()
    label = ds["train"].features["sentiment"].names[pred]
    return label


test_sentences = [
    "Thầy giảng bài rất dễ hiểu và nhiệt tình.",
    "Môn học này không hữu ích và tốn thời gian.",
    "Bài tập khó và không có hướng dẫn rõ ràng.",
    "Em rất thích cách dạy của cô giáo.",
    "Lớp học ồn ào và không hiệu quả.",
    "Bài tập quá ít, không đủ để thực hành."
]

for sent in test_sentences:
    print(f" \"{sent}\"\n Prediction: {predict_sentiment(sent)}\n")

 "Thầy giảng bài rất dễ hiểu và nhiệt tình."
 Prediction: positive

 "Môn học này không hữu ích và tốn thời gian."
 Prediction: negative

 "Bài tập khó và không có hướng dẫn rõ ràng."
 Prediction: negative

 "Em rất thích cách dạy của cô giáo."
 Prediction: positive

 "Lớp học ồn ào và không hiệu quả."
 Prediction: negative

 "Bài tập quá ít, không đủ để thực hành."
 Prediction: negative

